# Ebay Web-Scraping President Day Sales
# Qihan Guan


Write code that saves all item URLs to the file "deals.txt".

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import mysql.connector
import warnings
import json
import codecs


headers = {'User-Agent':'Mozilla/5.0'}  

#initialize a list to save the item urls 
deals_out = []

#Loop through the first 10 pages 
for i in range(1,11):
    URL= "https://www.ebay.com/e/daily-deals/hiw-presidents-day-deals-white-sale?_pgn="+str(i)
    page = requests.get(URL, headers=headers) 
    doc = BeautifulSoup(page.content, 'html.parser')
    #Look for item url
    item_link = doc.find_all('a', class_ = 's-item__link')
    for item in item_link:
        item_url = item.get('href')
        print(item_url)
        deals_out.append(item_url)
    time.sleep(4)    

        

https://www.ebay.com/itm/Egyptian-Comfort-1800-Count-4-Piece-Bed-Sheet-Set-Deep-Pocket-Bed-Sheets/142436469971?epid=1760442729&hash=item2129e00cd3%3Ag%3A7gIAAOSw3YBdRVJd&_trkparms=%2526rpp_cid%253D601435485fceeb223c6f4511&var=442541824291
https://www.ebay.com/itm/Super-Deluxe-1800-Count-Hotel-Quality-4-Piece-Deep-Pocket-Bed-Sheet-Set/122583280722?hash=item1c8a886c52%3Ag%3AmyMAAOSwnEtbOm07&_trkparms=%2526rpp_cid%253D601435485fceeb223c6f4511&var=424996915341
https://www.ebay.com/itm/Egyptian-Comfort-3-PC-Duvet-Cover-Set-1800-Count-Ultra-Soft-Cover-for-Comforter/143169152146?hash=item21558be492%3Ag%3A-6IAAOSw3AFchpif&_trkparms=%2526rpp_cid%253D601435485fceeb223c6f4511&var=442541824410
https://www.ebay.com/itm/1800-Pillow-Case-Set-Standard-or-King-Ultra-Soft-Pillowcase-Set-of-2-Pillowcases/111736675898?hash=item1a04065e3a%3Ag%3A5icAAOSwgMxdWvLp&_trkparms=%2526rpp_cid%253D601435485fceeb223c6f4511&var=414442331860
https://www.ebay.com/itm/Sherpa-Flannel-Fleece-Reversible-Blanket-Extra-Soft-B

In [ ]:
#Check the total number of URLs saved
len(deals_out)

216

In [ ]:
import os
#Write output to file 'deals.txt'
with open(os.getcwd()+'/deals.txt', 'w+') as filehandle:
    for item_url in deals_out:
        filehandle.write('%s\n' % item_url)


Write a program that opens the file saved and downloads each of the pages (URLs) into the folders "deals". Each file will be named as "item-id.html"

In [ ]:
import os
#read file and save the contents
deals = open(os.getcwd()+'/deals.txt', 'r').read().split('\n')
headers = {'User-Agent':'Mozilla/5.0'}  

#create path for 'deals' folder
path = os.path.join(os.getcwd() , 'deals')
if not os.path.exists(path):
    os.makedirs(path)

try:
    #Download each of the pages(URLs) into the folder 'deals'
    for URL in deals[0:216]: #escape the last empty row in deals.txt
        item_id = re.search(r'.+/(\d{12})\?', URL).group(1)
        page = requests.get(URL, headers=headers) 
        with open(os.path.join(path, str(item_id)+'.html'), 'w+', encoding="utf-8") as filehandle:
            filehandle.write(page.text)

        #put a 2 sec pause between queries
        time.sleep(2)
except:
    print('Problem with connection...')
        
    


In [ ]:
#Check how many files saved successfully
list = os.listdir(path) 
print(len(list))

184


184 files saved successfully. 

  
Write a separate piece of code that loops through the pages downloaded and opens and parses them into a Python or Java xxxxsoup-object. Identify and select: seller name, seller score, item price, item price, list price, # items sold, title, returns allowed(true / false), shipping price, condition (e.g., used, new, like new, seller refurbished, ...).

In [ ]:
#initialize a data frame
import pandas as pd
import numpy as np

deal_info = pd.DataFrame(columns=['Seller Name', 'Seller Score', 'Item Price', 'List Price', 'Number of Items Sold',
                                 'Title', 'Returns Allowed', 'Shipping Price', 'Condition'])

for i in range(0, len(list)):
    try:
        page = list[i]
        page_file = open(path + '\\' + page, 'r', encoding="utf-8").read() 
        soup = BeautifulSoup(page_file, 'html.parser')

        #selector command for seller name 
        seller_name = soup.find('div', class_ = 'mbg vi-VR-margBtm3').find('span', class_ = 'mbg-nw').get_text()

        #selector command for seller score
        seller_score = soup.find('span', class_ = 'mbg-l').find_next().get_text()

        #selector command for item price
        item_price_raw = soup.find('span', class_ = 'notranslate', itemprop = 'price').get_text()
        item_price = round(float(re.search(r'.*\$([\.\d]+)', item_price_raw).group(1)),2)

        #selector command for list price (note that some items do not have list price)
        if soup.find('span', id = 'orgPrc', class_ = 'notranslate ms-orp'):
            list_price_raw = soup.find('span', id = 'orgPrc', class_ = 'notranslate ms-orp').get_text()
            list_price = round(float(re.search(r'.*\$([\.\d]+)', list_price_raw).group(1)),2)
        else:
            list_price = None #if item does not have list price, set it to null

        #selector command for number of items sold (note that some items do not have # sold)
        if soup.find('a', class_ = 'vi-txt-underline'):
            num_sold_raw = soup.find('a', class_ = 'vi-txt-underline').get_text()
            num_sold = re.search(r'(.+) sold', num_sold_raw).group(1)
            num_sold = int(num_sold.replace(',',''))
        else:
            num_sold = 0

        #selector command for title
        title_raw = soup.find('h1', class_ = 'it-ttl', id = 'itemTitle').get_text()
        title = title_raw.replace('Details about  \xa0', '')
        title

        #selector command for returns allowed(true/false)
        return_policy = soup.find('span', id = 'vi-ret-accrd-txt').get_text()
        return_allowed = False if return_policy == 'Seller does not accept returns' else True

        #selector command for shipping price

        ##Note that free shipping can be standard or 'FAST N' Free'.
        ##It's easier to check for valid shipping price. If no $ was found, then set shipping price to 0.
        shipping_price = round(0.00,2)
        #Check for shipping price tag
        if soup.find('div', id = 'shippingSummary').find('span', id = 'fshippingCost'):
            ship_pc = soup.find('div', id = 'shippingSummary').find('span', id = 'fshippingCost').find_next().get_text()
            #See if price contains $ sign
            if (ship_pc.find('$') != -1):
                #if contains $, set shipping price, else set price to 0
                shipping_price = round(float(re.search(r'.*\$(.+)', ship_pc).group(1)),2)
        else:
            shipping_price = round(0.00,2)

        #selector command for condition
        condition = soup.find('div', id = 'vi-itm-cond').get_text()

        #save this iteration's result to the i-th row of the dataframe
        deal_info.loc[i] = [seller_name, seller_score, item_price, list_price, num_sold, title, 
                            return_allowed, shipping_price, condition]
    except:
        print('Error with: ' + page)

In [ ]:
#The saved data frame
deal_info

,Seller Name,Seller Score,Item Price,List Price,Number of Items Sold,Title,Returns Allowed,Shipping Price,Condition
0,cozyarray,219912,7.99,16.99,2791,SET OF 1500 TC PILLOWCASES TWO PILLOW CASES PE...,True,0.0,New with tags
1,cozyarray,219912,7.99,17.99,14865,1200 SERIES PILLOWCASES - 2 Pillow Cases Per S...,True,0.0,New with tags
2,cozyarray,219912,21.99,79.99,14504,1800 Count 4 Piece Deep Pocket Bed Sheet Set -...,True,0.0,New with tags
3,cozyarray,219912,5.99,29.99,4192,Bedskirt Pins - Set of 8 Plastic Head Bed Skir...,True,0.0,New with tags
4,cozyarray,219912,5.99,29.99,27877,1800 Pillow Case Set Standard or King Ultra So...,True,0.0,New with tags
...,...,...,...,...,...,...,...,...,...
179,www.powersellerusa.com,77178,14.99,23.39,406,Checked Window Curtain Drape Plaid Gingham Che...,True,0.0,New
180,howplumb,146985,14.97,38.99,72,2 Blackout Window Curtains Panel Pair Grommet ...,True,0.0,New
181,www.powersellerusa.com,77178,12.99,75.96,189,Supreme Super Soft 4 Piece Bed Sheet Set Deep ...,True,0.0,New with tags
182,dealgenius,712387,14.00,25,33,Blankie Tails Original Mermaid Tail Blanket Ki...,True,0.0,New with tags



Connect to MySQL. Save the information of items into a single table named "deals"in “msba”. 

In [ ]:
import mysql.connector
import warnings
warnings.filterwarnings("ignore")
SQL_DB = "msba"

#intialize a copy of the data frame and update it with processed values during the loop 
#to prepare for summary stats in (3)(f) 

deal_info1 = pd.DataFrame(None, index = deal_info.index,
                          columns=['Seller Name', 'Seller Score', 'Item Price', 'List Price', 'Number of Items Sold',
                                 'Title', 'Returns Allowed', 'Shipping Price', 'Condition', 'Have List Price'])

def main():
    
    SQL_TABLE_DEALS = "deals"
    SQL_TABLE_DEALS_DEF = "(" + \
            "seller_name VARCHAR(100) NULL" + \
            ", seller_score INT NULL" + \
            ", item_price INT NULL" + \
            ", list_price INT NULL" + \
            ", num_sold INT NULL" + \
            ", title VARCHAR(200) NULL" + \
            ", returns_allowed BOOLEAN NULL" + \
            ", shipping_price INT NULL" + \
            ", item_condition VARCHAR(100) NULL" + \
            ")"
    
    #create table "deals"
    create_sql_table(SQL_TABLE_DEALS, SQL_TABLE_DEALS_DEF)
    
    #insert values row by row into table "deals", missing values will be saved as NULL
    for i in deal_info.index:
        try:
            seller_name = str(deal_info['Seller Name'][i])
        except:
            seller_name = None
        try:
            seller_score = int(deal_info['Seller Score'][i])
        except:
            seller_score = None
        try:
            item_price = int(deal_info['Item Price'][i]*100)
        except: 
            item_price = None
        try:
            list_price = int(deal_info['List Price'][i]*100)
        except:
            list_price = None
        try:
            num_sold = int(deal_info['Number of Items Sold'][i])
        except:
            num_sold = None
        try:
            title = str(deal_info['Title'][i])
        except:
            title = None
        try:
            returns_allowed = deal_info['Returns Allowed'][i]
        except: 
            returns_allowed = None
        try:
            shipping_price = int(deal_info['Shipping Price'][i]*100)
        except:
            shipping_price = None
        try:
            item_condition = str(deal_info['Condition'][i])
        except:
            item_condition = None
          
        #update the dataframe for summary stats analysis in (3)(f)
        deal_info1['Seller Name'][i] = seller_name
        deal_info1['Seller Score'][i] = seller_score
        deal_info1['Item Price'][i] = item_price
        deal_info1['List Price'][i] = list_price
        deal_info1['Number of Items Sold'][i] = num_sold
        deal_info1['Title'][i] = title
        deal_info1['Returns Allowed'][i] = returns_allowed
        deal_info1['Shipping Price'][i] = shipping_price
        deal_info1['Condition'][i] = item_condition
        deal_info1['Have List Price'][i] = 0 if list_price == None else 1
        
        #for each iteration, insert one row into table "deals"
        try:
        
            #connect to local SQL instance
            conn = mysql.connector.connect(host='localhost',
                                            user='root',
                                            password='1234')
            cursor = conn.cursor()

            #query to insert row into table 
            entries = "(seller_name, seller_score, item_price, list_price, num_sold, title, returns_allowed, shipping_price, item_condition)"
            values = (seller_name, seller_score, item_price, list_price, num_sold, title, returns_allowed, shipping_price, item_condition)        
            query = "INSERT INTO " + SQL_DB + "." + SQL_TABLE_DEALS + " " + entries + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) ;"
            cursor.execute(query, values)
            conn.commit()
            cursor.close()
            conn.close()
            print("Row " + str(i) + " successsfully inserted") 
        
        except:
            print("Problem with row " + str(i))
            
        
    
# function definition for table creation sql 
def create_sql_table(SQL_TABLE_DEALS, SQL_TABLE_DEALS_DEF):
    try:
        
        #connect to local SQL instance
        conn = mysql.connector.connect(host='localhost',
                                            user='root',
                                            password='1234')
        cursor = conn.cursor()
        
        #create the table "deals" in "msba" 
        query = "CREATE TABLE IF NOT EXISTS " + SQL_DB + "." + SQL_TABLE_DEALS + " " + SQL_TABLE_DEALS_DEF + ";";
        print("Table created")
        cursor.execute(query);
        cursor.close()
        conn.close()
        return

    except IOError as e:
        print(e)

if __name__ == "__main__":
    main()

Table created
Row 0 successsfully inserted
Row 1 successsfully inserted
Row 2 successsfully inserted
Row 3 successsfully inserted
Row 4 successsfully inserted
Row 5 successsfully inserted
Row 6 successsfully inserted
Row 7 successsfully inserted
Row 8 successsfully inserted
Row 9 successsfully inserted
Row 10 successsfully inserted
Row 11 successsfully inserted
Row 12 successsfully inserted
Row 13 successsfully inserted
Row 14 successsfully inserted
Row 15 successsfully inserted
Row 16 successsfully inserted
Row 17 successsfully inserted
Row 18 successsfully inserted
Row 19 successsfully inserted
Row 20 successsfully inserted
Row 21 successsfully inserted
Row 22 successsfully inserted
Row 23 successsfully inserted
Row 24 successsfully inserted
Row 25 successsfully inserted
Row 26 successsfully inserted
Row 27 successsfully inserted
Row 28 successsfully inserted
Row 29 successsfully inserted
Row 30 successsfully inserted
Row 31 successsfully inserted
Row 32 successsfully inserted
Row 33


Run summary stats on each item. Print to the screen the mean, min, max, and mean for each numeric column, dependent on (i) whether “list price”was providedand (ii) "condition" (group by at the same time, not separately).

In [ ]:
#Use the dataframe deal_info1 obatained from (3)(e) to perform summary stats analysis
summary_df = deal_info1[['Seller Score', 'Item Price', 'List Price', 'Number of Items Sold', 'Shipping Price','Have List Price', 'Condition']]

#Convert numeric columns to numeric for analysis
convert_dict = {'Seller Score':int, 'Item Price':int,
                'List Price':int,'Number of Items Sold':int, 
                'Shipping Price':int}
summary_df = summary_df.convert_dtypes(convert_dict)

#Summary stats 
summary_df.groupby(['Have List Price', 'Condition']).agg(['mean', 'median', 'min', 'max'])

Seller Score                         \
                                        mean  median    min     max   
Have List Price Condition                                             
0               New             37965.750000   21453  20539   99658   
                New with tags   60907.555556   22367    177  420348   
1               New             86685.000000   77178  65499  146985   
                New with tags  194955.880000  219912   3197  799174   

                                Item Price                      List Price  \
                                      mean median  min    max         mean   
Have List Price Condition                                                    
0               New            1566.000000   1049  550   3999          NaN   
                New with tags  2710.333333   2299  599  13499          NaN   
1               New            2321.272727   1499  999   9499  7703.363636   
                New with tags  2365.533333   1998  599   7981  8007.226667   

                                                  Number of Items Sold  \
                              median   min    max                 mean   
Have List Price Condition                                                
0               New             <NA>  <NA>   <NA>          1977.750000   
                New with tags   <NA>  <NA>   <NA>          1379.566667   
1               New             3899  2339  40595          3429.363636   
                New with tags   7998  1698  24999          7201.786667   

                                                  Shipping Price             \
                               median min     max           mean median min   
Have List Price Condition                                                     
0               New            1470.5  59    6273              0      0   0   
                New with tags   781.5   0    6979              0      0   0   
1               New             406.0   0   28135              0      0   0   
                New with tags   629.0   0  181453              0      0   0   

                                   
                              max  
Have List Price Condition          
0               New             0  
                New with tags   0  
1               New             0  
                New with tags   0


The existence of ‘list price’ does appear to influence sales positively. From the summary stats, we can see that the median and mean item price for items with list price and items without list price are relatively close. However, the mean number of items sold for items that have list price is much higher than those without list price. We can roughly conclude that the existence of ‘list price’ appears to increase sales. 
